# Carregando dados

In [0]:
import pandas as pd

In [0]:
uri = "https://raw.githubusercontent.com/SrDiegoH/DataScience/master/machine-learning-otimizacao-de-modelos-atraves-de-hiperparametros-e-exploracao-aleatoria/machine-learning-carros-simulacao.csv"
carros_df = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [0]:
carros_df.head(7)

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290
5,55405.26,1,11,23594.53374
6,93415.61,1,16,16077.30660


# Manipulando dados

## Gerado dados aleatorios

In [0]:
import numpy as np

In [0]:
SEED = 158020

np.random.seed(SEED)

## Dividindo dados com train test split

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [0]:
x = carros_df[["preco", "idade_do_modelo","km_por_ano"]]
y = carros_df["vendido"]

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

print(f"{len(treino_x)} dados de treino e {len(teste_x)} dados de teste")

7500 dados de treino e 2500 dados de teste


## Treinando com Dummy Classifier

In [0]:
from sklearn.dummy import DummyClassifier

In [0]:
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)

acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print(f"Acurácia do Dummy Classifier: {acuracia.round(2)}")

Acurácia do Dummy Classifier: 50.96


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


## Treinando com Decision Tree

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [0]:
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)

previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100

print(f"Acurácia do DecisionTree: {acuracia.round(2)}")

Acurácia do DecisionTree: 71.92


## Utilizando DecisionTree com Cross Validate

Estudar hiperparametros em [machine-learning-otimizacao-de-modelos-atraves-de-hiperparametros-e-exploracao-aleatoria](https://raw.githubusercontent.com/SrDiegoH/DataScience/master/machine-learning-otimizacao-de-modelos-atraves-de-hiperparametros-e-exploracao-aleatoria/machine-learning-otimizacao-de-modelos-atraves-de-hiperparametros-e-exploracao-aleatoria.ipynb)

In [0]:
from sklearn.model_selection import cross_validate

In [0]:
SEED = 301

np.random.seed(SEED)

In [0]:
modelo = DecisionTreeClassifier(max_depth=2)

results = cross_validate(modelo, x, y, cv=3, return_train_score=False)

media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print("Acurácia com Cross Validate: 3 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Acurácia com Cross Validate: 3 = [74.99, 76.57]


Aumentando o Cross Validate

In [0]:
modelo = DecisionTreeClassifier(max_depth=2)

results = cross_validate(modelo, x, y, cv=5, return_train_score=False)

media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print("Acurácia com Cross Validate: 5 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Acurácia com Cross Validate: 5 = [75.21, 76.35]


Aumentando mais ainda o Cross Validate

In [0]:
modelo = DecisionTreeClassifier(max_depth=2)

results = cross_validate(modelo, x, y, cv=10, return_train_score=False)

media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print("Acurácia com Cross Validate: 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Acurácia com Cross Validate: 10 = [74.24, 77.32]


## Gerando aleatoriedade no Cross validate

In [0]:
def imprime_resultados(results):
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()

  print("Acurácia média: %.2f" % (media * 100))
  print("Intervalo de acurácia: [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

In [0]:
from sklearn.model_selection import KFold

Sem embaralhar

In [0]:
cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=2)

results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.78
Intervalo de acurácia: [74.37, 77.19]


Embaralhando

In [0]:
cv = KFold(n_splits = 10, shuffle = True)

modelo = DecisionTreeClassifier(max_depth=2)

results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.78
Intervalo de acurácia: [72.64, 78.92]


# Situação em que as classes estão proporcionalmente desbalanceadas

In [0]:
carros_desbalanceado_df = carros_df.sort_values("vendido", ascending=True)

x_desbalanceado = carros_desbalanceado_df[["preco", "idade_do_modelo","km_por_ano"]]
y_desbalanceado = carros_desbalanceado_df["vendido"]

carros_desbalanceado_df.head(7)

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704
5310,62474.09,0,19,26142.11896
5309,78095.40,0,17,31834.35454


Sem embaralhar

In [0]:
cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=2)

results = cross_validate(modelo, x_desbalanceado, y_desbalanceado, cv=cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 57.84
Intervalo de acurácia: [34.29, 81.39]


Embaralhando

In [0]:
cv = KFold(n_splits = 10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)

results = cross_validate(modelo, x_desbalanceado, y_desbalanceado, cv=cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.78
Intervalo de acurácia: [72.88, 78.68]


## Testando validação com StratifiedKFold

* https://scikit-learn.org/stable/modules/cross_validation.html

In [0]:
from sklearn.model_selection import StratifiedKFold

In [0]:
cv = StratifiedKFold(n_splits = 10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)

results = cross_validate(modelo, x_desbalanceado, y_desbalanceado, cv=cv, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.79
Intervalo de acurácia: [73.68, 77.90]


# Como evitar vicios na validação?

## Gerando dados aleatórios para simular de agrupamento quando usar os estimadores

In [0]:
np.random.seed(SEED)

carros_df['modelo'] = carros_df.idade_do_modelo + np.random.randint(-2, 3, size=10000)
carros_df.modelo = carros_df.modelo + abs(carros_df.modelo.min()) + 1
carros_df.head(7)

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5
5,55405.26,1,11,23594.53374,13
6,93415.61,1,16,16077.30660,20


In [0]:
carros_df.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [0]:
carros_df.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

## Testando validação com GroupKFold

In [0]:
from sklearn.model_selection import GroupKFold

In [0]:
cv = GroupKFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_desbalanceado, y_desbalanceado, cv=cv, groups=carros_df.modelo, return_train_score=False)

imprime_resultados(results)

Acurácia média: 75.78
Intervalo de acurácia: [73.67, 77.90]


## Cross validação utilizando StandardScaler

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [0]:
scaler = StandardScaler()
scaler.fit(treino_x)

treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)

previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100

print(f"Acurácia de {acuracia.round(2)}")

Acurácia foi 74.4


In [0]:
cv = GroupKFold(n_splits = 10)

modelo = SVC()

results = cross_validate(modelo, x_desbalanceado, y_desbalanceado, cv=cv, groups=carros_df.modelo, return_train_score=False)

imprime_resultados(results)

Acurácia média: 77.27
Intervalo de acurácia: [74.35, 80.20]


In [0]:
scaler = StandardScaler()
scaler.fit(x_desbalanceado)

x_desbalanceado_escalado = scaler.transform(x_desbalanceado)

In [0]:
cv = GroupKFold(n_splits = 10)

modelo = SVC()

results = cross_validate(modelo, x_desbalanceado_escalado, y_desbalanceado, cv=cv, groups = carros_df.modelo, return_train_score=False)

imprime_resultados(results)

Acurácia média: 76.70
Intervalo de acurácia: [74.30, 79.10]


### Gerando Pipeline

In [0]:
from sklearn.pipeline import Pipeline

In [0]:
scaler = StandardScaler()

modelo = SVC()

pipeline = Pipeline([('transformacao', scaler),
                     ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)

results = cross_validate(pipeline, x_desbalanceado, y_desbalanceado, cv=cv, groups=carros_df.modelo, return_train_score=False)

imprime_resultados(results)

Accuracy médio: 76.68
Accuracy intervalo: [74.28, 79.08]
